In [7]:
from pytz import timezone
from datetime import datetime

TZ = timezone('Europe/Moscow')

In [3]:
from env_simple_move import HumanMoveSimpleAction


env = HumanMoveSimpleAction(render_mode = 'human', continuous=True)
seed = int( datetime.now(TZ).strftime("%H%M%S") )
print(seed)
env.action_space.seed(seed)

total_reward = 0.
step_reward = []

observation, info = env.reset(seed=seed)
# course move, speed


for tick in range(800):

    action = env.teach_action(observation)

    observation, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        observation, info = env.reset()
        step_reward.append(reward)
        total_reward += reward
        print('BREAK')
        break
    
    step_reward.append(reward)
    total_reward += reward

print(len(step_reward))
print(total_reward)

164346
BREAK
60
543.123456535397


In [4]:
env.close()

In [5]:
import pygame
import math
import vector 



zero = vector.obj(x=300, y=300)

screen_width = zero.x * 2
screen_height = zero.y * 2
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("Moving")

black = (0, 0, 0)
white = (255, 255, 255)
red = (255, 0, 0)
green = (0, 255, 0)
blue = (0, 0, 255)




while True:

    screen.fill(black)


    pygame.draw.circle(screen, green, (zero.x, zero.y), 3)

    # рисуем треугольник
    course = math.radians(90)

    triangle = zero
    triangle_width = 10
    triangle_height = 10

    corner2 = vector.obj(x= triangle.x + triangle_width / 2, y= triangle.y + triangle_height) - zero 
    corner3 = vector.obj(x= triangle.x - triangle_width / 2, y= triangle.y + triangle_height) - zero

    corner2 = corner2.rotateZ(course) + zero
    corner3 = corner3.rotateZ(course) + zero

    triangle_points = [(triangle.x, triangle.y), (corner2.x, corner2.y),
                    (corner3.x, corner3.y)]
    
    pygame.draw.polygon(screen, red, triangle_points)

    pygame.draw.circle(screen, blue, (zero.x + 150, zero.y - 90), 10)

    pygame.draw.circle(screen, white, (zero.x - 150, zero.y + 90), 20)

    pygame.display.update()

    # устанавливаем частоту обновления экрана
    pygame.time.Clock().tick(60)

    b_quit = False
    for event in pygame.event.get():
        
        if event.type == pygame.QUIT:
            b_quit = True
        if event.type == pygame.MOUSEBUTTONDOWN: # останавливаем падение звезд по клику
            freeze = True
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_ESCAPE:
                b_quit = True
            if event.key == pygame.K_RETURN: # возобновляем движение вниз, если нажат Enter
                freeze = False

    if b_quit == True:
        break

pygame.quit()

In [2]:
pygame.quit()

In [18]:
from PIL import Image, ImageDraw
import numpy as np

black = (0, 0, 0)
white = (255, 255, 255)
red = (255, 0, 0)
green = (0, 255, 0)
blue = (0, 0, 255)
 
# Пустой желтый фон.
im = Image.new('RGB', (600, 600), (0, 0, 0))
draw = ImageDraw.Draw(im)
 
draw.ellipse((100, 100, 110, 110), fill='blue', outline='blue')
 
draw.polygon(
    xy=(
        (200, 200),
        (300, 100),
        (250, 50)
    ), fill='red', outline='red'
)

draw.point((450, 450), fill=green)

inp = np.asarray(im)

im.save('draw-render.jpg', quality=95)

TypeError: ImageDraw.point() got an unexpected keyword argument 'width'

In [1]:
from env_simple_move import HumanMoveSimpleAction
import numpy as np
from stable_baselines3 import DQN
from typing import Any, Dict, Tuple, Union
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.logger import Video
from stable_baselines3.common.logger import KVWriter, Logger
import torch
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

DQN_path = "../sb3_DQN/"

tmp_path = DQN_path + "model_1"

env = HumanMoveSimpleAction(render_mode = 'rgb_array', continuous=False)
model = DQN.load(tmp_path, env)

logger = Logger(folder = DQN_path, output_formats=["stdout", "log", "json", "csv"])

#Video
screens = []

def grab_screens(_locals: Dict[str, Any], _globals: Dict[str, Any]) -> None:
    """
    Renders the environment in its current state, recording the screen in the captured `screens` list

    :param _locals: A dictionary containing all local variables of the callback's scope
    :param _globals: A dictionary containing all global variables of the callback's scope
    """
    screen = env.render()
    screens.append(screen.transpose(2, 0, 1))

evaluate_policy(
    model,
    env,
    callback=grab_screens,
    n_eval_episodes=1,
    deterministic=True,
)

logger.record(
    "video",
    Video(torch.from_numpy(np.asarray([screens])), fps=30),
    #exclude=("stdout", "log", "json", "csv"),
)

r:\Scool\Anaconda\envs\rl_conda_311\Lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


In [9]:
from env_simple_move import HumanMoveSimpleAction
import numpy as np
from stable_baselines3 import DQN
from typing import Any, Dict, Tuple, Union
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.logger import Video
from stable_baselines3.common.logger import KVWriter, Logger
import torch
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

env_id = "MoveSimple"
video_folder = "logs/videos/"
video_length = 30 * 10


DQN_path = "../sb3_DQN/"

tmp_path = DQN_path + "model"

seed = int( datetime.now(TZ).strftime("%H%M%S") )
env = HumanMoveSimpleAction(render_mode = 'rgb_array', continuous=False, seed = seed)
read_model = DQN.load(tmp_path, env)

vec_env = DummyVecEnv([lambda: env])

# Record the video starting at the first step
vec_env = VecVideoRecorder(vec_env, video_folder,
                       record_video_trigger=lambda x: x == 0, video_length=video_length,
                       name_prefix=f"DQN-agent-{env_id}")


observation = vec_env.reset()
t1 = False
t2 = False
t3 = False
for _ in range(video_length + 1):
  action, _ = read_model.predict(observation)
  observation, t1, t2, t3 = vec_env.step(action)
  if t2[0] == True:
    break
# Save the video
vec_env.close()

Moviepy - Building video r:\Scool\Otus\HomeWork\otus_final\source\env\logs\videos\random-agent-MoveSimple-step-0-to-step-300.mp4.
Moviepy - Writing video r:\Scool\Otus\HomeWork\otus_final\source\env\logs\videos\random-agent-MoveSimple-step-0-to-step-300.mp4



Moviepy - Done !
Moviepy - video ready r:\Scool\Otus\HomeWork\otus_final\source\env\logs\videos\random-agent-MoveSimple-step-0-to-step-300.mp4
